In [12]:
import sqlite3
import requests
import time
import os
from dotenv import load_dotenv

In [3]:
conn = sqlite3.connect("seq_data.db")
cur = conn.cursor()

In [4]:
cur.execute("""
CREATE TABLE IF NOT EXISTS dim_company (
	cik INTEGER PRIMARY KEY,
	cik_padded TEXT NOT NULL,
	ticker TEXT,
	company_name TEXT NOT NULL
);
""")

In [13]:
load_dotenv()
user_agent = os.getenv('user_agent')

Ivo Boon ivo@ivoboon.nl
